In [1]:
from search.Problem import Problem
from search.Search import aStarSearch, breadthFirstSearch, ipa_star

# L'algorithme A*
A* est un algorithme de recherche permettant de trouver le chemin dans un graphe entre un noeud de départ et un noeud d'arrivé.
Cette algorithme repose sur deux éléments importants:
 - Un problème formalisé
 - Une heuristique admissible et consistante
 
L'algorithme consiste à explorer les noeuds du graphes en passant par les meilleurs noeuds, que l'on classe selon leur valeur $f(n) = g(n) + h(n)$ où $g(n)$ est la profondeur du noeud $n$ et $h(n)$ l'heuristique au noeud $n$.

## Formalisation du problème
Il est nécessaire de réfléchir à la formalisation du problème que l'on souhaite résoudre.

### La tour de Hanoï
Prenons le jeu de la tour de Hanoï et essayons de le modéliser afin de la résoudre à l'aide de l'algorithme A*.
Le principe est simple, on dispose de trois emplacements qui peuvent accueillir des disques de tailles différentes. Il faut déplacer la tour de gauche sur l'emplacement de droite avec le moins de coups possibles. Chaque disque ne peut être posé que sur un disque de diamètre supérieur au sien.

In [2]:
import copy

class Hanoi(Problem):
    def __init__(self):
        start = [[i for i in reversed(range(5))], [], []]
        end = [[], [], [i for i in reversed(range(5))]]
        super().__init__(start, end)
        
    def _buildNext(self, prev):
        children = []
        
        for i in range(3):
            board = copy.deepcopy(prev)

            if len(board[i]) > 0:
                piece = board[i].pop()
                
                possible_moves = [0, 1, 2]
                possible_moves.remove(i)
                
                for j in possible_moves:
                    if len(board[j]) == 0 or board[j][-1] > piece:
                        next_board = copy.deepcopy(board)
                        next_board[j].append(piece)
                        children.append(next_board)
                
        return children    

## Heuristique
Une heuristique est une fonction qui, pour un noeud donné, donne le coût permettant d'atteindre le noeud but.

On dit d'une heuristique qu'elle est **admissible** lorsqu'elle est inférieure ou égale au coût réel pour se rendre au but.
On dit également d'une heuristique qu'elle est **consistante** (ou monotone) lorsque, pour chaque nœud $n$ et chaque successeur $n’$ de $n$ produit par n’importe quelle action, le coût estimé pour atteindre le but à partir de $n$ n’est pas supérieur au coût de l’étape pour aller à $n’$ plus le coût estimé pour atteindre le but à partir de $n’$.

Tout l'art des algorithmes de recherche réside dans le développement d'heuristiques admissibles et consistantes afin de préserver l'optimalité de l'algorithme.

L'utilisation de certaines heuristiques dites triviales équivaut à l'utilisation d'algorithmes de recherches dérivés:
- Si $f(n) = g(n)$: algorithme du coût uniforme, qui équivaut à une exploration en largeur d'abord lorsque les coûts pour passer d'un noeud parent à un noeud fils sont égaux (Breadth First Search)
- Si $h(n) = 0 \Rightarrow f(n) = g(n)$: algorithme glouton (Best first search)

Créons une heuristique pour notre problème. Commençons simplement par poser $h(n) = 0$, de cette façon, l'algorithme agira comme un algorithme glouton et va tester toutes les possibilités en parcourant le graphe des coups en largeur. On trouvera ainsi assurément la solution la plus courte (en nombre de coups) mais au détriment du temps de recherche.

In [3]:
def heuristic(state, goal):
    return 0

## Résolution
Voyons le résultat de notre algorithme

In [4]:
problem = Hanoi()
result = aStarSearch(problem, heuristic)

print(result)

Solution found in 0.051723000000000074 seconds
Length of the solution: 31
Nodes opened: 233


On voit donc que l'algorithme a trouvé le chemin le plus court (le jeu de la tour de Hanoï est bien connu, il est admis que la tour de Hanoïe à 5 disques ne peut se résoudre en moins de 31 mouvements).

En cherchant la solution, l'algorithme a parcouru 233 noeuds du graphes des coups jouables.

## Une autre heuristique ?
Cherchons maintenant une heuristique un peu plus intelligente pour estimer la distance entre un noeud et le noeud que l'on cherche à atteindre.

On peut admettre que le nombre de coup qu'il reste à jouer est inférieur au nombre de pièces sur la position 3

Posons donc $h(n) = 5 - \textit{nb_pieces_en_3}$

Ceci est une heuristique admissible, en effet, lorsqu'il ne reste plus qu'une seule pièce à déplacer l'heuristique proposée est égale au coût réel de déplacement. Lorsqu'il reste plus d'une pièce à déplacer, l'heuristique proposée est toujours inférieure au nombre de déplacement réel du fait de la mécanisme de déplacement ; un disque étant toujours posé sur un disque de diamètre plus grand, pour déplacer $n$ pièces, il faudra toujours plus de $n$ mouvements.

In [5]:
def wrong_place_heuristic(state, goal):
    return 5 - len(state.Value[2])

In [6]:
result2 = aStarSearch(problem, wrong_place_heuristic)
print(result2)

Solution found in 0.03436000000000006 seconds
Length of the solution: 31
Nodes opened: 178


On s'aperçoit qu'avec une heuristique plus "intelligente", l'algorithme présente de bien meilleur performance, il a parcouru 76% de noeuds en moins pour trouver la solution optimale en moins de temps.

## Encore un problème
Le jeu de la tour de Hanoï est un problème extrêmement simple et peut même être résolu rapidement par des algorithmes récursifs, mais il permet de tester l'efficacité des algorithmes de recherche.

Testons maintenant notre algorithme sur un problème régulièrement utilisé dans le domaine des jeux vidéos : la recherche de chemin dans un labyrinthe.
Définissons une structure de données simplifiée où chaque ligne du labyrinthe est définie par une chaine de caractères, chaque case correspond à un caractère de la chaine. Si le caractères est un espace, le passage est libre, si le caractères est #, c'est un mur et on ne peut pas passer.

In [94]:
class Maze(Problem):
    def __init__(self, maze):
        start = (4, 0)
        end = (1, 9)
        self.maze = maze
        super().__init__(start, end)
        
    def _buildNext(self, prev):
        children = []
        x, y = prev
        if x > 0 and self.maze[x-1][y] != '#':
            children.append((x-1, y))
        if x < len(self.maze)-1 and self.maze[x+1][y] != '#':
            children.append((x+1, y))
        if y > 0 and self.maze[x][y-1] != '#':
            children.append((x, y-1))
        if y < len(self.maze[0])-1 and self.maze[x][y+1] != '#':
            children.append((x, y+1))
        return children

Dans ce cas, l'heuristique la plus simple reste la distance entre les deux points. On peut utiliser la distance euclidienne, ou la distance de manhattan qui donnerait la distance exacte s'il n'y avait pas les obstacles.

In [95]:
def manhattan_heuristic(state, goal):
    x1, y1 = state.Value
    x2, y2 = goal.Value
    return abs(x1 - x2) + abs(y1 - y2)

In [96]:
my_maze = [
    '  ####    ',
    '     #   A',
    '  #  # # #',
    ' ###   # #',
    'D  ##  ###',
    '    ##    ',
]

maze_problem = Maze(my_maze)

In [97]:
result4 = aStarSearch(maze_problem, manhattan_heuristic)
print(result4)
print(result4.Path)

Solution found in 0.0008850000000002467 seconds
Length of the solution: 16
Nodes opened: 27
[(3, 0) (G=1, H=11), (2, 0) (G=2, H=10), (1, 0) (G=3, H=9), (1, 1) (G=4, H=8), (1, 2) (G=5, H=7), (1, 3) (G=6, H=6), (1, 4) (G=7, H=5), (2, 4) (G=8, H=6), (3, 4) (G=9, H=7), (3, 5) (G=10, H=6), (3, 6) (G=11, H=5), (2, 6) (G=12, H=4), (1, 6) (G=13, H=3), (1, 7) (G=14, H=2), (1, 8) (G=15, H=1), (1, 9) (G=16, H=0)]


L'algorithme trouve très rapidement le chemin le plus court dans le labyrinthe tout en évitant les obstacles (murs).

Il est "amusant" de noter que si l'on souhaite répondre à une problématique très légèrement différents, il est nécessaire de modifier la modélisation du problème.
Imaginons que l'on souhaite non pas uniquement relier deux points, mais que l'on souhaite passer par un point intermédiaire. Il devient nécessaire de revoir les états de départ et d'arrivé ainsi que la construction des noeuds enfants.

In [98]:
class MazeStep(Problem):
    def __init__(self, maze):
        start = ((4, 0), False)
        end = ((1, 9), True)
        self.step = (5, 0)
        self.maze = maze
        super().__init__(start, end)
        
    def _buildNext(self, prev):
        children = []
        xy, val_step = prev
        x, y = xy
        if x > 0 and self.maze[x-1][y] != '#':
            r_step = val_step
            if (x-1, y) == self.step:
                r_step = True
            child = ((x-1, y), r_step)
            children.append(child)
        if x < len(self.maze)-1 and self.maze[x+1][y] != '#':
            r_step = val_step
            if (x+1, y) == self.step:
                r_step = True
            child = ((x+1, y), r_step)
            children.append(child)
        if y > 0 and self.maze[x][y-1] != '#':
            r_step = val_step
            if (x, y-1) == self.step:
                r_step = True
            child = ((x, y-1), r_step)
            children.append(child)
        if y < len(self.maze[0])-1 and self.maze[x][y+1] != '#':
            r_step = val_step
            if (x, y+1) == self.step:
                r_step = True
            child = ((x, y+1), r_step)
            children.append(child)
        return children

In [99]:
def manhattan_heuristic(state, goal):
    x1, y1 = state.Value[0]
    x2, y2 = goal.Value[0]
    return abs(x1 - x2) + abs(y1 - y2)

In [100]:
step_problem = MazeStep(my_maze)
result5 = aStarSearch(step_problem, manhattan_heuristic)
print(result5)

Solution found in 0.003775000000000084 seconds
Length of the solution: 18
Nodes opened: 60


In [101]:
result5.Path

[((5, 0), True) (G=1, H=13),
 ((4, 0), True) (G=2, H=12),
 ((3, 0), True) (G=3, H=11),
 ((2, 0), True) (G=4, H=10),
 ((1, 0), True) (G=5, H=9),
 ((1, 1), True) (G=6, H=8),
 ((1, 2), True) (G=7, H=7),
 ((1, 3), True) (G=8, H=6),
 ((1, 4), True) (G=9, H=5),
 ((2, 4), True) (G=10, H=6),
 ((3, 4), True) (G=11, H=7),
 ((3, 5), True) (G=12, H=6),
 ((3, 6), True) (G=13, H=5),
 ((2, 6), True) (G=14, H=4),
 ((1, 6), True) (G=15, H=3),
 ((1, 7), True) (G=16, H=2),
 ((1, 8), True) (G=17, H=1),
 ((1, 9), True) (G=18, H=0)]